In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Oct_dict_JSON.json', 'r') as json_file:
    link_day_minute_Oct_dict_JSON = json.load(json_file)

In [10]:
week_day_Oct_list = [1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 15, 16, 17, 18, 19, 22, 23, 24, 25, 26, 29, 30, 31]

In [11]:
link_day_minute_Oct_list = []
for link_idx in range(24):
    for day in week_day_Oct_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Oct_list.append(link_day_minute_Oct_dict_JSON[key] ['NT_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Oct_list)

66240

In [13]:
x = np.matrix(link_day_minute_Oct_list)
x = np.matrix.reshape(x, 24, 2760)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2760)

In [15]:
x[:,:2]

matrix([[  9609.4709637 ,   9464.57872558],
        [  9659.80854965,  12042.05972524],
        [ 16526.94253948,  16128.15643082],
        [ 12728.30437374,  13733.35647954],
        [ 13168.07730515,  10711.32440986],
        [ 10990.95497126,   8050.66420588],
        [ 11457.03517662,   9969.94185202],
        [ 11136.44949084,  10386.47227524],
        [ 24715.30580829,  24683.33769506],
        [ 13003.96211757,  12706.30027655],
        [ 14532.56292107,  14243.85544923],
        [ 19259.38944657,  19028.10159113],
        [  7559.078271  ,   8194.11069879],
        [  7391.00847088,   7835.85356837],
        [ 14178.33527204,   7385.85876195],
        [  5832.21802828,   6953.11826129],
        [ 12954.96765318,  11565.59837623],
        [ 10226.94887995,   8920.05879341],
        [ 11616.01736932,  12676.28954443],
        [  6806.25101436,   8884.10156559],
        [ 17294.60541323,   9229.77280372],
        [  9404.28324638,   7736.75653935],
        [ 10129.92009943,   5689

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [5e-02, 6e+00]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.90183383e+06  0.00000000e+00  4.61e+03 1.90e+02  9.46e+05     0s
   1   1.65950783e+06 -2.67875644e+06  4.62e+02 9.89e+01  1.67e+05     0s
   2   4.18097484e+05 -1.00721696e+06  4.62e-04 1.14e-13  2.55e+04     0s
   3  -7.00875625e+04 -2.65848731e+05  4.62e-10 3.55e-14  3.5

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Oct_weekday_NT.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[1.2914406625694632e-09,
 14901.090800639044,
 1.451676415575905e-09,
 1154.9915642992526,
 2880.6564216341,
 2.9994737461763732e-05,
 4.318298997887655e-07,
 3026.5563524548575,
 3.0691531746724196e-09,
 10610.650570667558,
 3.0691516325269664e-09,
 3.0691525522124706e-09,
 3.068839321152288e-09,
 3.0864419815606955e-09,
 2.8920868174405617e-05,
 2.8812610987565684e-05,
 8.972717082656595e-06,
 1419.6403569976528,
 3743.9802400178364,
 18180.087333615513,
 2.9948142703756925e-05,
 0.0002510557241914291,
 10192.709363150743,
 5.2161140369606784e-05,
 4035.4150219383914,
 2461.9336796366915,
 2488.6801875188276,
 11138.509438661786,
 13246.159015380748,
 2274.304744683614,
 4.4290765047513836e-05,
 7801.373488313147,
 6997.717145741018,
 0.00045610707955862084,
 0.0003555226544046386,
 5228.612855388675,
 1379.7509713689608,
 7299.734857866466,
 2.4170899390742878e-05,
 8345.121313269898,
 1.3434340696566113e-05,
 1.3415307128065296e-05,
 1.9324570944861952e-05,
 1.92823654796212e-05,
 